# Stream2segment notebook examples

To (re)run this notebook with no modifications (e.g., using the example database below) on your computer, it is assumed that you opened it from the directory created with the `s2s init` command


## Table of contents
- [Database data overview](#databaseoverview)
    - [The Segment object](#segment_object)
    - [Related objects](#related_objects)
- [Working with ObsPy](#obspy)
- [Selection of segments via simplified expressions](#selexpr)
- [Selection expression cheatsheet](#selexpr_cheatsheet)
   - [Examples](#selexpr_examples)
   - [Selectable segment (and related objects) attributes](#selectable_attributes)

## Processing segments<a name="selexpr"></a>

Stream2segment has two main processing functions, i.e. functions whose purpose is to fetch segments from the database, run some custom code on them, and produce the desired output. These functions are `process` and `imap`:

In [1]:
from stream2segment.process import process, imap

In their base form, both functions accept the following arguments (type `help(process)` or `help(imap)` for details):

```python
process(pyfunc, dburl, segments_selection, config, output, ...)
```

```python
imap(pyfunc, dburl, segments_selection, config, ...)
```
where

- `pyfunc` is the processing fucntion (Python function): `pyfunc(segment, config)`
- `dburl` is the database (db) URL
- `segments_selection` is the selection of desired segments from the given db
- `config` is an optional dict to be passed to the processing function

```python
process(python_function, dburl, segments_selection, output)
```


In [2]:
help(imap)

Help on function imap in module stream2segment.process.main:

imap(pyfunc, dburl, segments_selection=None, config=None, logfile='', verbose=False, multi_process=False, chunksize=None, skip_exceptions=None)
    Return an iterator that applies the function `pyfunc` to every segment
    found on the database at the URL `dburl`, processing only segments matching
    the given selection (`segments_selection`), yielding the results in the form
    of the tuple:
    ```
        (output:Any, segment_id:int)
    ```
    (where output is the return value of `pyfunc`)
    
    :param pyfunc: a Python function with signature (= accepting arguments):
        `(segment:Segment, config:dict)`. The first argument is the segment
        object which will be automatically passed from this function
    :param dburl: the database URL. Supported formats are Sqlite and Postgres
        (See https://docs.sqlalchemy.org/en/14/core/engines.html#database-urls).
    :param segments_selection: The segments to be 

## Selection of segments via simplified expressions<a name="selexpr"></a>

Stream2segment offers the advantage of a simple selection of database segments with no need to dig into the functions of the [Object-Relational Mapping (ORM) Python library](https://www.sqlalchemy.org/) which is used under the hood to communicate with the database. The latter has the benefit of exposing each segment as a simple Python object called `Segment`. The selection of suitable segments is performed by creating a `dict` mapping one or more Segment attributes to a selection expression for that attribute: 

In [3]:
from stream2segment.process import Segment, get_segments
from sqlalchemy.orm import load_only

# The database URL should be the parameter 'dburl' of the download configuration file.
# Here we use an example database (2 segments) provided in the same directory of this notebook:
import os
dburl = 'sqlite:///' + os.path.join(os.getcwd(), 'jupyter.example.db')

# create the selection dict:
segments_selection = {
  'has_data': 'true',
  'maxgap_numsamples': '[-0.5, 0.5]',
  'event_distance_deg': '[20, 90]'
  # other optional attributes (see cheatsheet below for details):
  # missing_data_sec: '<120'
  # missing_data_ratio: '<0.5'
  # id: '<300'
  # event.time: "(2014-01-01T00:00:00, 2014-12-31T23:59:59)"
  # event.latitude: "[24, 70]"
  # event.longitude: "[-11, 24]"
}


for seg in get_segments(dburl, segments_selection):
    # seg is our Segment instance, let's print its id:
    print(seg.id)

1
2


## Segment selection cheatsheet<a name='selexpr_cheatsheet'></a>

As described above, each segment is exposed to the user as a simple Python object with the following attributes. Each attribute can be used to select specific segments, as illustrateed above, or to access the segment properties:

### Syntax <a name='selexpr_examples'></a>

```python
segments_selection: {
  "[attribute]": "[expression]",
  "[attribute]": "[expression]",
  ...
}
```
(segments_selection is just a convention, as any Python variable name you can use the one you want)

Example:

1. Not all segments on the database have waveform data. This might happen for several reason (server error, connection error and so on). As such, **in most cases you might probably want to work with segments with waveform data** (at least one byte of data):
```python
{"has_data": "true"}
```
2. To select and work on segments of stations activated in 2017 only:
```python
{"station.start_time": "[2017-01-01, 2018-01-01T00:00:00)"}
```
(brackets denote intervals. Square brackets include end-points, round brackets exclude endpoints)

3. To select segments from specified ids, e.g. 1, 4, 342, 67 (e.g., ids which raised errors during
   a previous run and whose id where logged might need inspection in the GUI):
   segment_select:
```python
{"id": "1 4 342 67"}
```

4. To select segments whose event magnitude is greater than 4.2:
```python
{"event.magnitude": ">4.2"}
```
(the same way work the operators: =, >=, <=, <, !=)

5. To select segments with a particular channel sensor description:
```python
{"channel.sensor_description": "'GURALP CMG-40T-30S'"}
```
(note: for attributes with str values and spaces, we need to quote twice, as otherwise
"GURALP CMG-40T-30S" would match 'GURALP' and 'CMG-40T-30S', but not the whole string.
See attribute types below)


### Selectable segment (and related objects) attributes <a name='selectable_attributes'></a>


Attribute                    | Python type and (optional) description
:----------------------------|:-------------------------------------------------------------
id                           | int: segment (unique) db id
event_distance_deg           | float: distance between the segment's station and<br>the event, in degrees
event_distance_km            | float: distance between the segment's station and<br>the event, in km, assuming a perfectly spherical earth<br>with a radius of 6371 km
start_time                   | datetime.datetime: the waveform data start time
arrival_time                 | datetime.datetime: the station's arrival time of the waveform.<br>Value between 'start_time' and 'end_time'
end_time                     | datetime.datetime: the waveform data end time
request_start                | datetime.datetime: the requested start time of the data
request_end                  | datetime.datetime: the requested end time of the data
duration_sec                 | float: the waveform data duration, in seconds
missing_data_sec             | float: the number of seconds of missing data, with respect<br>to the requested time window. It might also be negative<br>(more data received than requested). This parameter is useful<br>when selecting segments: e.g., if we requested 5<br>minutes of data and we want to process segments with at<br>least 4 minutes of downloaded data, then:<br>missing_data_sec: '< 60'
missing_data_ratio           | float: the portion of missing data, with respect<br>to the request time window. It might also be negative<br>(more data received than requested). This parameter is useful<br>when selecting segments: e.g., if you want to process<br>segments whose real time window is at least 90% of the<br>requested one, then: missing_data_ratio: '< 0.1'
sample_rate                  | float: the waveform data sample rate.<br>It might differ from the segment channel's sample_rate
has_data                     | boolean: tells if the segment has data saved (at least<br>one byte of data). This parameter is useful when selecting<br>segments (in most cases, almost necessary), e.g.:<br>has_data: 'true'
download_code                | int: the code reporting the segment download status. This<br>parameter is useful to further refine the segment selection<br>skipping beforehand segments with malformed data (code -2):<br>has_data: 'true'<br>download_code: '!=-2'<br>(All other codes are generally of no interest for the user.<br>However, for details see Table 2 in<br>https://doi.org/10.1785/0220180314#tb2)
maxgap_numsamples            | float: the maximum gap or overlap found in the waveform data,<br>in number of points. If 0, the segment has no gaps/overlaps.<br>Otherwise, if >=1: the segment has gaps, if <=-1: the segment<br>has overlaps. Values in (-1, 1) are difficult to interpret: a<br>rule of thumb is to consider half a point a gap / overlap<br>(maxgap_numsamples > 0.5 or maxgap_numsamples < -0.5).<br>This parameter is useful when selecting segments: e.g.,<br>to select segments with no gaps/overlaps, then:<br>maxgap_numsamples: '(-0.5, 0.5)'
seed_id                      | str: the seed identifier in the typical format<br>[Network].[Station].[Location].[Channel]. For segments<br>with waveform data, `data_seed_id` (see below) might be<br>faster to fetch.
data_seed_id                 | str: same as 'segment.seed_id', but faster to get because it<br>reads the value stored in the waveform data. The drawback<br>is that this value is null for segments with no waveform data
has_class                    | boolean: tells if the segment has (at least one) class<br>assigned
data                         | bytes: the waveform (raw) data. Used by `segment.stream()`
-----------------------------| ------------------------------------------------
event                        | object (attributes below)
event.id                     | int
event.event_id               | str: the id returned by the web service or catalog
event.time                   | datetime.datetime
event.latitude               | float
event.longitude              | float
event.depth_km               | float
event.author                 | str
event.catalog                | str
event.contributor            | str
event.contributor_id         | str
event.mag_type               | str
event.magnitude              | float
event.mag_author             | str
event.event_location_name    | str
-----------------------------| ------------------------------------------------
channel                      | object (attributes below)
channel.id                   | int
channel.location             | str
channel.channel              | str
channel.depth                | float
channel.azimuth              | float
channel.dip                  | float
channel.sensor_description   | str
channel.scale                | float
channel.scale_freq           | float
channel.scale_units          | str
channel.sample_rate          | float
channel.band_code            | str: the first letter of channel.channel
channel.instrument_code      | str: the second letter of channel.channel
channel.orientation_code     | str: the third letter of channel.channel
channel.station              | object: same as segment.station (see below)
-----------------------------| ------------------------------------------------
station                      | object (attributes below)
station.id                   | int
station.network              | str: the station's network code, e.g. 'AZ'
station.station              | str: the station code, e.g. 'NHZR'
station.netsta_code          | str: the network + station code, concatenated with<br>the dot, e.g.: 'AZ.NHZR'
station.latitude             | float
station.longitude            | float
station.elevation            | float
station.site_name            | str
station.start_time           | datetime.datetime
station.end_time             | datetime.datetime
station.has_inventory        | boolean: tells if the segment's station inventory has<br>data saved (at least one byte of data).<br>This parameter is useful when selecting segments: e.g.,<br>to select only segments with inventory downloaded:<br>station.has_inventory: 'true'
station.datacenter           | object (same as segment.datacenter, see below)
-----------------------------| ------------------------------------------------
datacenter                   | object (attributes below)
datacenter.id                | int
datacenter.station_url       | str
datacenter.dataselect_url    | str
datacenter.organization_name | str
-----------------------------| ------------------------------------------------
download                     | object (attributes below): the download execution
download.id                  | int
download.run_time            | datetime.datetime
-----------------------------| ------------------------------------------------
classes.id                   | int: the id(s) of the classes assigned to the segment
classes.label                | int: the label(s) of the classes assigned to the segment
classes.description          | int: the description(s) of the classes assigned to the<br>segment

### The Segment object <a name='segment_object'></a>

`get_segments`, when called with a string URL, opens a database session, yields selected segments and closes the session afterwards automatically. A [database session](https://docs.sqlalchemy.org/en/13/orm/session_basics.html) establishes all conversations with the
database and represents a "holding zone"  for all the objects which you’ve loaded or associated with it during
its lifespan. Closing a session is recommended after you finished your work as it releases memory on the computer and (if the db is remote) on the server, avoiding potential problems. After a session is closed, all segment objects are **detached** from the database, which means we can not access anymore all of its properties, but only those previously loaded.

Calling `get_segments` and working within a loop is usually fine. However, in some cases (as in the remainder of this notebook), you might want to keep the session open. In this case, simply call `get_segments` with a session object instead of the `dburl`. The session can be manually closed at the end of your work:

In [20]:
from stream2segment.process import get_session, close_session
session = get_session(dburl)
# do your work ...
close_session(session)

True

So, let's open a new session and fetch the first segment, lewaving the session open

In [21]:
from stream2segment.process import get_session
seg = list(get_segments(session, segments_selection))[0]
# close_session(session)  # <- (commented out, we do not want to close the session yet)

Let's inspect the session object

In [22]:
print(str(seg))

Segment
 attributes (17 of 17 loaded):
  data_seed_id: GE.RUE..BH (str, 11 characters, showing first 10 only)
  event_distance_deg: 88.40960422432707 (float)
  data: b'808468D RU' (bytes, 9216 elements, showing first 10 only)
  download_code: 200 (int)
  start_time: 2017-09-08 05:00:00.495000 (datetime)
  arrival_time: 2017-09-08 05:02:05.252870 (datetime)
  end_time: 2017-09-08 05:04:12.245000 (datetime)
  sample_rate: 20.0 (float)
  maxgap_numsamples: 0.0 (float)
  request_start: 2017-09-08 05:00:05 (datetime)
  request_end: 2017-09-08 05:04:05 (datetime)
  queryauth: False (bool)
  event_id: 1 (int)
  channel_id: 2 (int)
  datacenter_id: 1 (int)
  download_id: 2 (int)
  id: 1 (int)
 related_objects (0 of 6 loaded):
  event
  channel
  classes
  datacenter
  download
  station


Each attribute can be accessed as simple Python attribute:

In [23]:
arrival_time = seg.arrival_time  # datetime.datetime object
print('Segment P-wave arrival time is %s' % str(seg.arrival_time))

Segment P-wave arrival time is 2017-09-08 05:02:05.252870


Other accessible attributes not shown above are (for a complete list see [below](#selectable_attributes)):

In [24]:
print(seg.maxgap_numsamples)
print(seg.missing_data_sec)
print(seg.missing_data_ratio)
print(seg.duration_sec)
print(seg.has_data)
print(seg.event_distance_km)

0.0
-11.75
-0.04895833333333344
251.75
True
9830.699456398519


### Related objects <a name='related_objects'></a>

Each Segment has also a set of related Python objects easily accessible. Such a nice feature would be extremely complex to implement without a database, with waveforms and metadata stored as files on your comoputer. Let's inspect some related objects (these objects are loaded into memory only upon access, and therefore we need the db session to be open):

`Event` object:

In [26]:
evt = seg.event
print(str(evt))

Event
 attributes (16 of 16 loaded):
  event_id: 20170908_0 (str, 16 characters, showing first 10 only)
  time: 2017-09-08 04:49:21.200000 (datetime)
  latitude: 15.02 (float)
  longitude: -93.81 (float)
  depth_km: 72.0 (float)
  author: EMSC (str)
  catalog: EMSC-RTS (str)
  contributor: EMSC (str)
  contributor_id: 616600 (str)
  mag_type: mw (str)
  magnitude: 8.1 (float)
  mag_author: EMSC (str)
  event_location_name: OFFSHORE C (str, 24 characters, showing first 10 only)
  event_type: None (NoneType)
  webservice_id: 1 (int)
  id: 1 (int)
 related_objects (0 of 1 loaded):
  segments


Note that the event has the back-reference `segments`, which is a list of Segment objects because by design one segment is always related to one event, whereas one event generates many recordings at different stations, and thus is related to many segments. (be aware of potential memory problems when accessing huge lists of related objects. For details, see the section [Selection of segments via simplified expressions](#selexpr)).

The same kind of "segments relation" holds for boith the `Station` and `Channel` objects (see below for details).

`Station` object:

In [10]:
sta = seg.station
print(str(sta))

Station
 attributes (11 of 11 loaded):
  network: GE (str)
  station: RUE (str)
  latitude: 52.4759 (float)
  longitude: 13.78 (float)
  elevation: 40.0 (float)
  site_name: None (NoneType)
  start_time: 2012-03-21 10:00:00 (datetime)
  end_time: None (NoneType)
  inventory_xml: b'\x1f\x8b\x08\x00\xa4\x99\x1b\\\x02\xff' (bytes, 44710 elements, showing first 10 only)
  datacenter_id: 1 (int)
  id: 2 (int)
 related_objects (0 of 3 loaded):
  datacenter
  channels
  segments


Other accessible attributes not shown above are (for a complete list see [below](#selectable_attributes)):

In [11]:
print(sta.netsta_code)

GE.RUE


`Channel` object:

In [12]:
cha = seg.channel
print(str(cha))

Channel
 attributes (12 of 12 loaded):
  location:  (str)
  channel: BHZ (str)
  depth: 3.0 (float)
  azimuth: 0.0 (float)
  dip: -90.0 (float)
  sensor_description: GFZ:GE1993 (str, 25 characters, showing first 10 only)
  scale: 588000000.0 (float)
  scale_freq: 0.02 (float)
  scale_units: M/S (str)
  sample_rate: 20.0 (float)
  station_id: 2 (int)
  id: 2 (int)
 related_objects (0 of 2 loaded):
  station
  segments


Other accessible attributes not shown above are (for a complete list see [below](#selectable_attributes)):

In [13]:
print(cha.band_code)
print(cha.instrument_code)
print(cha.orientation_code)

B
H
Z


## Working with ObsPy<a name='obspy'></a>

Each segment waveform data is stored as bytes sequence in the `segment.data` attribute. However, you seldom need to access this attribute directly: `Stream2segment` defines shortcut methods to work with the relative ObsPy Objects.

For instance, let's access the the ObsPy `Stream` representing the waveform data of our `seg` object fetched above:

In [14]:
# Get ObsPy Stream object
stream = seg.stream()
# a Stream is a collection of traces, let's take the first one and inspect it:
trace = stream[0]
print('Trace data: ' + str(trace.data))

Trace data: [  196   211    94 ..., -2008 -1464 -1010]


Now let's remove the instrumental response:

In [15]:
# Get ObsPy Inventory object:
inventory = seg.inventory()
# remove the response:
stream_remresp = stream.remove_response(inventory)
trace_rr = stream_remresp[0]
print('Trace data (response removed): ' + str(trace_rr.data))

Trace data (response removed): [  5.82973528e-07   5.26137453e-07   5.78937133e-07 ...,  -7.80069136e-07
  -1.14732931e-06  -6.83767823e-07]


<b>Caveat</b>: The trace data has now been permanently modified. This is not due to Stream2segment but to a specific design choice of ObsPy. **In other words, `segment.stream()` from now returns `stream_remresp`** (the stream with the response removed!):

In [16]:
stream = seg.stream()
trace = stream[0]
print('Trace data:' + str(trace.data))

Trace data:[  5.82973528e-07   5.26137453e-07   5.78937133e-07 ...,  -7.80069136e-07
  -1.14732931e-06  -6.83767823e-07]


Note that, similar to `remove_response`, several other `Stream` and `Trace` methods permanently modify the underlying data (please refer to their ObsPy documentation before applying them). In all of these cases, to recover the original trace, there are two strategies:
<p>
 1] Reload the segment stream from the database with <code>segment.stream(reload=True)</code>

In [17]:
stream = seg.stream(reload=True)
trace= stream[0]
print('Trace data:' + str(trace.data))

Trace data:[  196   211    94 ..., -2008 -1464 -1010]


2] (<i>recommended</i>) Preserve <code>segment.stream()</code> using remove_response on a stream copy

In [18]:
stream_remresp = stream.copy().remove_response(inventory)
trace = seg.stream()[0]
print('Trace data:' + str(trace.data))

Trace data:[  196   211    94 ..., -2008 -1464 -1010]


**Caveat** When working with lists of objects, like the `query` object above, because `Stream2segment` is designed for massive downloads, it is better to load only each object id, deferring the download of all other attributes upon access: this is what `.options(load_only('id'))` above does (note that "id" is an attribute common to all objects types: `Segment` , `Event`, `Station`, and so on).

We suggest to use the same approach for loading lists of related objects, e.g.:

In [19]:
evt = seg.event
# load event related segments (*risk of memory overflow: low):
segments = evt.segments.options(load_only('id')).all()

cha = seg.channel
# load channel related segments (*risk of memory overflow: medium):
segments = cha.segments.options(load_only('id')).all()

sta = seg.station
# load station related segments (*risk of memory overflow: high):
segments = sta.segments.options(load_only('id')).all()

dct = seg.datacenter
# load data center (e.g. IRIS) related segments (*risk of memory overflow: very high):
segments = dct.segments.options(load_only('id')).all()

\* The levels of risk reported are just heuristically estimated and have to be considered reliable only relative to each other (an event has almost certainly less related segments than a channel, which has almost certainly less related segments than a station, and so on)

***In any case, for really memory consuming or slow tasks, consider moving the Notebook code into a custom Python module and use the command `s2s process`, which is specifically designed to better manage memory and performance***